In [1]:
import pandas as pd 

In [2]:
from bs4 import BeautifulSoup

In [3]:
def open_link(year_str): 
#create a function that takes 1 directory which is the year of the factbook and leads to a link of a webpage and returns that webpage as a string
    page = open('factbook/'+year_str+'/docs/notesanddefs.html', encoding = "utf8").read()
    #read the entire webpage from local files as a string
    return page

In [4]:
page = open_link('2016')
page = BeautifulSoup(page) #create a BeautifulSoup object from the string above
print(page.prettify()[:500]) #view how the tags are nested

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<!--<![endif]-->
<html class="no-js" lang="en">
 <!-- InstanceBegin template="/Templates/wfbext_template.dwt.cfm" codeOutsideHTMLIsLocked="false" -->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-


In [5]:
def find_path(string): #define the find_path function that will return column names and file paths by request
    cols = page.select('span.category') #creating a list of all of the span tags with the class category
    for col in cols: #iterate over each of the span tags
        cells = col.select('td') #use 'select' to find the 'td' tags inside the span
        col_name = cells[0].text #The first 'td' tag will give us the name of the column 
        link1 = cells[1].select('a')
        if len(link1) > 0:
            fpath = link1[0]['href'] #the second 'td' tag will have the path to the file contained in the href attribute
        if string in col_name: 
            print(col_name, fpath)
            #if the column name has the specified strings, print the path to the file and the column name

In [10]:
col_lst = ['Population', 'Area','Birth rate','Climate','Coastline','Death rate',
           'GDP (purchasing power parity)','Health expenditures','Government type',
           'Infant mortality rate','Inflation rate','Internet users','Land use','Literacy',
           'Net migration', 'GDP - composition, by sector of origin']
#a list of all the columns whose paths need to be found to retrieve data

In [11]:
for name in col_lst: #iterate through each element in the col_lst list
    a = find_path(name)
    #find the path of each element 

Population ../fields/2119.html#184
Population below poverty line ../fields/2046.html#185
Population distribution ../fields/2267.html
Population growth rate ../fields/2002.html#186
Population pyramid ../fields/2002.html#186
Area ../fields/2147.html#10
Area - comparative ../fields/2023.html#11
Birth rate ../fields/2054.html#13
Climate ../fields/2059.html#22
Coastline ../fields/2060.html#23
Death rate ../fields/2066.html#41
GDP (purchasing power parity) ../fields/2001.html#82
Health expenditures ../fields/2225.html#103
Government type ../fields/2128.html#96
Infant mortality rate ../fields/2091.html#118
Inflation rate (consumer prices) ../fields/2092.html#119
Internet users ../fields/2153.html#126
Land use ../fields/2097.html#134
Literacy ../fields/2103.html#139
Net migration rate ../fields/2112.html#170
GDP - composition, by sector of origin ../fields/2012.html#84


In [12]:
dict_col = {'Area':'2147.html','Birth_rate':'2054.html','Climate':'2059.html',
            'Coastline':'2060.html','Death_rate':'2066.html','GDP_composition_by_sector_of_origin':'2012.html',
            'GDP_(purchasing power parity)':'2001.html','Government_type':'2128.html',
            'Health_expenditures':'2225.html','Infant_mortality_rate':'2091.html',
            'Inflation_rate':'2092.html','Internet_users':'2153.html','Land_use':'2097.html','Literacy':'2103.html',
            'Net_migration':'2112.html', 'Population':'2119.html' }
# a dictionary of the columns and their innermost directory in their links 

In [13]:
def make_lst_row(x): #define a function that extracts the data from the table rows of a table in a file and append them to a list
    rows = x.find_all('tr') #find all table rows and produce a list of them
    lst_row = [] #create an empty list
    for row in rows: #iterate over each of the element in the rows list
        cell = row.find_all('td') #find all the table cells in a table row and produce a list of cells
        cell = str(cell) #turn the cells of each row into a string
        clean = BeautifulSoup(cell,'lxml').get_text()
        #pass the 'cell' string into BeautifulSoup to create a Beautiful soup object and extract the text without html tags using get_text()
        lst_row.append(clean) #append all the cleantext strings into the list lst_row
    return(lst_row) #return lst_row

In [14]:
def make_df(x): #create a function that produces a dataframe for a beautifulsoup object by request 
    lst_row = make_lst_row(x) #call the make_lst_row function defined above
    x_df = pd.DataFrame(lst_row) #create a dataframe from the list lst_row
    x_df = x_df[0].str.split(', \n', expand = True)
    #split the "0" column into two columns by splitting the string in each row
    x_df[0] = x_df[0].str.lstrip('[') 
    #get rid of the '[' at the beginning of the string in each row in column "0"
    x_df[1] = x_df[1].str.rstrip('\n]') 
    #get rid of the '\n]' at the end of the string in each row in column "1"
    col_label = x.find_all('th')
    #find the table headers for the dataframe by finding all the table headers in the internet object
    all_header = [] #create an empty list
    col_label = str(col_label)
    clean = BeautifulSoup(col_label, 'lxml').get_text()
    #pass the 'col_label' string into BeautifulSoup to create a BeautifulSoup object and extract the text without html tags using get_text()
    all_header.append(clean) #append clean to the all_header list
    df = pd.DataFrame(all_header) #create a dataframe from the all_header list
    df = df[0].str.split(', ', expand=True)
    #split the "0" column into two columns
    df[0] = df[0].str.lstrip('[') #get rid of the '[' at the beginning of the string in the first column
    df[1] = df[1].str.rstrip(']') #get rid of the ']' at the end of the string in the second column
    frame = [df,x_df]
    new_df = pd.concat(frame) #concatenate two dataframes into a single dataframe
    new_df = new_df.rename(columns=new_df.iloc[0]).drop([0])
    #assign the first row to be the table header and drop the "0" rows
    return new_df

In [20]:
dict_1 = {} 
#create a list that would contain the column names and their corresponding beautifulsoup objects that were retrieved from the links found above
for key in dict_col: #iterate through the keys in the dict_col dictionary above 
    if key not in dict_1: 
        dict_1[key] = ''
    #if each key in the dict_col dictionary is not in the dict_1 dictionary, add to the dictionary and assign them the empty string value
    col_name = open('factbook/2016/fields/'+ dict_col[key], encoding='utf8').read()
    #read the html webpage of each column as a string
    col_name = BeautifulSoup(col_name) #create a beautifulsoup object 
    col_name.prettify() #view how the tags are nested
    dict_1[key] = col_name #assign each beautifulsoup object that is created to the corresponding key in dict_1

In [21]:
lst_key = list(dict_1.keys()) #create a list of keys of dict_1 

In [22]:
df1 = make_df(dict_1[lst_key[0]])
df2 = make_df(dict_1[lst_key[1]])
df3 = make_df(dict_1[lst_key[2]])
df4 = make_df(dict_1[lst_key[3]])
df5 = make_df(dict_1[lst_key[4]])
df6 = make_df(dict_1[lst_key[5]])
df7 = make_df(dict_1[lst_key[6]])
df8 = make_df(dict_1[lst_key[7]])
df9 = make_df(dict_1[lst_key[8]])
df10 = make_df(dict_1[lst_key[9]])
df11 = make_df(dict_1[lst_key[10]])
df12 = make_df(dict_1[lst_key[11]])
df13 = make_df(dict_1[lst_key[12]])
df14 = make_df(dict_1[lst_key[13]])
df15 = make_df(dict_1[lst_key[14]])
df16 = make_df(dict_1[lst_key[15]])
#for each of the key in the lst_key list, enter them as the beautifulsoup object that needs to be transformed into
#a dataframe

In [23]:
lst_df = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16]
#create a list of all the 16 dataframes that are the attributes of countries around the world

In [24]:
#use the while loop to merge all of the dataframes together
i = 2 #set the initial loop number at 2
final_df = pd.merge(lst_df[0],lst_df[1], on='Country', how='outer')
#set the first merge of the first two dataframes. Merge based on the 'Country column' and outer join
while i < len(lst_df): #while the loop number is less than the length of the lst_df list, continue the loop
    final_df = pd.merge(final_df,lst_df[i], on = 'Country', how='outer')
    #merge the new dataframe above with the next dataframe in the list  
    i = i + 1 #after each merge, increase i which indicates the next position in the lst_df list
final_df.head() #after merging, show the first five rows of the final final_df dataframe

,Country,AREA(SQ KM),"BIRTH RATE(BIRTHS/1,000 POPULATION)",CLIMATE,COASTLINE(KM),"DEATH RATE(DEATHS/1,000 POPULATION)",GDP - COMPOSITION,BY SECTOR OF ORIGIN(%)],GDP (PURCHASING POWER PARITY),GOVERNMENT TYPE,HEALTH EXPENDITURES(% OF GDP),"INFANT MORTALITY RATE(DEATHS/1,000 LIVE BIRTHS)",INFLATION RATE (CONSUMER PRICES)(%),INTERNET USERS,LAND USE(%),LITERACY(%),"NET MIGRATION RATE(MIGRANT(S)/1,000 POPULATION)",POPULATION
0,Afghanistan,"total: 652,230 sq km\nland: 652,230 sq km\nwat...","38.3 births/1,000 population (2016 est.)",arid to semiarid; cold winters and hot summers,0 km (landlocked),"13.7 deaths/1,000 population (2016 est.)",agriculture: 24%\nindustry: 21%\nservices: 55%...,NaN,$62.32 billion (2015 est.)\n$61.53 billion (20...,presidential Islamic republic,8.2% of GDP (2014),"total: 112.8 deaths/1,000 live births\nmale: 1...",-1.5% (2015 est.)\n4.6% (2014 est.),total: 2.69 million\npercent of population: 8....,agricultural land: 58.1%\narable land 11.9%; p...,definition: age 15 and over can read and write...,"-1.2 migrant(s)/1,000 population (2016 est.)","33,332,025 (July 2016 est.)"
1,Akrotiri,total: 123 sq km\nnote: includes a salt lake a...,NaN,"temperate; Mediterranean with hot, dry summers...",56.3 km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"approximately 15,700 on the Sovereign Base Are..."
2,Albania,"total: 28,748 sq km\nland: 27,398 sq km\nwater...","13.1 births/1,000 population (2016 est.)","mild temperate; cool, cloudy, wet winters; hot...",362 km,"6.7 deaths/1,000 population (2016 est.)",agriculture: 21.9%\nindustry: 14.9%\nservices:...,NaN,$32.65 billion (2015 est.)\n$31.81 billion (20...,parliamentary republic,5.9% of GDP (2014),"total: 12.3 deaths/1,000 live births\nmale: 13...",1.9% (2015 est.)\n1.6% (2014 est.),total: 1.916 million\npercent of population: 6...,agricultural land: 43.8%\narable land 22.7%; p...,definition: age 15 and over can read and write...,"-3.3 migrant(s)/1,000 population (2016 est.)","3,038,594 (July 2016 est.)"
3,Algeria,"total: 2,381,741 sq km\nland: 2,381,741 sq km\...","23 births/1,000 population (2016 est.)","arid to semiarid; mild, wet winters with hot, ...",998 km,"4.3 deaths/1,000 population (2016 est.)",agriculture: 13.1%\nindustry: 39%\nservices: 4...,NaN,$578.7 billion (2015 est.)\n$557.8 billion (20...,presidential republic,7.2% of GDP (2014),"total: 20.3 deaths/1,000 live births\nmale: 21...",4.8% (2015 est.)\n2.9% (2014 est.),total: 15.105 million\npercent of population: ...,agricultural land: 17.3%\narable land 3.1%; pe...,definition: age 15 and over can read and write...,"-0.9 migrant(s)/1,000 population (2016 est.)","40,263,711 (July 2016 est.)"
4,American Samoa,total: 199 sq km\nland: 199 sq km\nwater: 0 sq...,"22.9 births/1,000 population (2016 est.)","tropical marine, moderated by southeast trade ...",116 km,"4.8 deaths/1,000 population (2016 est.)",agriculture: 27.4%\nindustry: 12.4%\nservices:...,NaN,$711 million (2013 est.)\n$718 million (2012 e...,presidential democracy; a self-governing terri...,NaN,"total: 8.5 deaths/1,000 live births\nmale: 10....",2.1% (2013)\n3.5% (2012),"total: 17,000\npercent of population: 31.3% (J...",agricultural land: 21.7%\narable land 13.3%; p...,NaN,"-20.6 migrant(s)/1,000 population (2016 est.)","54,194 (July 2016 est.)"


In [25]:
final_df = final_df.dropna(axis=1, how='all')
#drop the 'BY SECTOR OF ORIGIN(%)]' column which only consists of NaN values
final_df.head()

,Country,AREA(SQ KM),"BIRTH RATE(BIRTHS/1,000 POPULATION)",CLIMATE,COASTLINE(KM),"DEATH RATE(DEATHS/1,000 POPULATION)",GDP - COMPOSITION,GDP (PURCHASING POWER PARITY),GOVERNMENT TYPE,HEALTH EXPENDITURES(% OF GDP),"INFANT MORTALITY RATE(DEATHS/1,000 LIVE BIRTHS)",INFLATION RATE (CONSUMER PRICES)(%),INTERNET USERS,LAND USE(%),LITERACY(%),"NET MIGRATION RATE(MIGRANT(S)/1,000 POPULATION)",POPULATION
0,Afghanistan,"total: 652,230 sq km\nland: 652,230 sq km\nwat...","38.3 births/1,000 population (2016 est.)",arid to semiarid; cold winters and hot summers,0 km (landlocked),"13.7 deaths/1,000 population (2016 est.)",agriculture: 24%\nindustry: 21%\nservices: 55%...,$62.32 billion (2015 est.)\n$61.53 billion (20...,presidential Islamic republic,8.2% of GDP (2014),"total: 112.8 deaths/1,000 live births\nmale: 1...",-1.5% (2015 est.)\n4.6% (2014 est.),total: 2.69 million\npercent of population: 8....,agricultural land: 58.1%\narable land 11.9%; p...,definition: age 15 and over can read and write...,"-1.2 migrant(s)/1,000 population (2016 est.)","33,332,025 (July 2016 est.)"
1,Akrotiri,total: 123 sq km\nnote: includes a salt lake a...,NaN,"temperate; Mediterranean with hot, dry summers...",56.3 km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"approximately 15,700 on the Sovereign Base Are..."
2,Albania,"total: 28,748 sq km\nland: 27,398 sq km\nwater...","13.1 births/1,000 population (2016 est.)","mild temperate; cool, cloudy, wet winters; hot...",362 km,"6.7 deaths/1,000 population (2016 est.)",agriculture: 21.9%\nindustry: 14.9%\nservices:...,$32.65 billion (2015 est.)\n$31.81 billion (20...,parliamentary republic,5.9% of GDP (2014),"total: 12.3 deaths/1,000 live births\nmale: 13...",1.9% (2015 est.)\n1.6% (2014 est.),total: 1.916 million\npercent of population: 6...,agricultural land: 43.8%\narable land 22.7%; p...,definition: age 15 and over can read and write...,"-3.3 migrant(s)/1,000 population (2016 est.)","3,038,594 (July 2016 est.)"
3,Algeria,"total: 2,381,741 sq km\nland: 2,381,741 sq km\...","23 births/1,000 population (2016 est.)","arid to semiarid; mild, wet winters with hot, ...",998 km,"4.3 deaths/1,000 population (2016 est.)",agriculture: 13.1%\nindustry: 39%\nservices: 4...,$578.7 billion (2015 est.)\n$557.8 billion (20...,presidential republic,7.2% of GDP (2014),"total: 20.3 deaths/1,000 live births\nmale: 21...",4.8% (2015 est.)\n2.9% (2014 est.),total: 15.105 million\npercent of population: ...,agricultural land: 17.3%\narable land 3.1%; pe...,definition: age 15 and over can read and write...,"-0.9 migrant(s)/1,000 population (2016 est.)","40,263,711 (July 2016 est.)"
4,American Samoa,total: 199 sq km\nland: 199 sq km\nwater: 0 sq...,"22.9 births/1,000 population (2016 est.)","tropical marine, moderated by southeast trade ...",116 km,"4.8 deaths/1,000 population (2016 est.)",agriculture: 27.4%\nindustry: 12.4%\nservices:...,$711 million (2013 est.)\n$718 million (2012 e...,presidential democracy; a self-governing terri...,NaN,"total: 8.5 deaths/1,000 live births\nmale: 10....",2.1% (2013)\n3.5% (2012),"total: 17,000\npercent of population: 31.3% (J...",agricultural land: 21.7%\narable land 13.3%; p...,NaN,"-20.6 migrant(s)/1,000 population (2016 est.)","54,194 (July 2016 est.)"


In [26]:
final_df.columns = ['Country', 'Area','Birth rate','Climate','Coastline','Death rate',
                    'GDP_composition','GDP_PPP','Government type','Health expenditures',
                    'Infant mortality rate','Inflation rate',
                    'Internet users','Land use','Literacy','Net migration', 'Population']
#shorten the names of all the columns in the final_df dataframe

In [27]:
final_df = final_df.fillna('0') #replace the NaN values with the 0 value
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,GDP_composition,GDP_PPP,Government type,Health expenditures,Infant mortality rate,Inflation rate,Internet users,Land use,Literacy,Net migration,Population
0,Afghanistan,"total: 652,230 sq km\nland: 652,230 sq km\nwat...","38.3 births/1,000 population (2016 est.)",arid to semiarid; cold winters and hot summers,0 km (landlocked),"13.7 deaths/1,000 population (2016 est.)",agriculture: 24%\nindustry: 21%\nservices: 55%...,$62.32 billion (2015 est.)\n$61.53 billion (20...,presidential Islamic republic,8.2% of GDP (2014),"total: 112.8 deaths/1,000 live births\nmale: 1...",-1.5% (2015 est.)\n4.6% (2014 est.),total: 2.69 million\npercent of population: 8....,agricultural land: 58.1%\narable land 11.9%; p...,definition: age 15 and over can read and write...,"-1.2 migrant(s)/1,000 population (2016 est.)","33,332,025 (July 2016 est.)"
1,Akrotiri,total: 123 sq km\nnote: includes a salt lake a...,0,"temperate; Mediterranean with hot, dry summers...",56.3 km,0,0,0,0,0,0,0,0,0,0,0,"approximately 15,700 on the Sovereign Base Are..."
2,Albania,"total: 28,748 sq km\nland: 27,398 sq km\nwater...","13.1 births/1,000 population (2016 est.)","mild temperate; cool, cloudy, wet winters; hot...",362 km,"6.7 deaths/1,000 population (2016 est.)",agriculture: 21.9%\nindustry: 14.9%\nservices:...,$32.65 billion (2015 est.)\n$31.81 billion (20...,parliamentary republic,5.9% of GDP (2014),"total: 12.3 deaths/1,000 live births\nmale: 13...",1.9% (2015 est.)\n1.6% (2014 est.),total: 1.916 million\npercent of population: 6...,agricultural land: 43.8%\narable land 22.7%; p...,definition: age 15 and over can read and write...,"-3.3 migrant(s)/1,000 population (2016 est.)","3,038,594 (July 2016 est.)"
3,Algeria,"total: 2,381,741 sq km\nland: 2,381,741 sq km\...","23 births/1,000 population (2016 est.)","arid to semiarid; mild, wet winters with hot, ...",998 km,"4.3 deaths/1,000 population (2016 est.)",agriculture: 13.1%\nindustry: 39%\nservices: 4...,$578.7 billion (2015 est.)\n$557.8 billion (20...,presidential republic,7.2% of GDP (2014),"total: 20.3 deaths/1,000 live births\nmale: 21...",4.8% (2015 est.)\n2.9% (2014 est.),total: 15.105 million\npercent of population: ...,agricultural land: 17.3%\narable land 3.1%; pe...,definition: age 15 and over can read and write...,"-0.9 migrant(s)/1,000 population (2016 est.)","40,263,711 (July 2016 est.)"
4,American Samoa,total: 199 sq km\nland: 199 sq km\nwater: 0 sq...,"22.9 births/1,000 population (2016 est.)","tropical marine, moderated by southeast trade ...",116 km,"4.8 deaths/1,000 population (2016 est.)",agriculture: 27.4%\nindustry: 12.4%\nservices:...,$711 million (2013 est.)\n$718 million (2012 e...,presidential democracy; a self-governing terri...,0,"total: 8.5 deaths/1,000 live births\nmale: 10....",2.1% (2013)\n3.5% (2012),"total: 17,000\npercent of population: 31.3% (J...",agricultural land: 21.7%\narable land 13.3%; p...,0,"-20.6 migrant(s)/1,000 population (2016 est.)","54,194 (July 2016 est.)"


In [28]:
final_df['Area'] = final_df['Area'].str.extract('(\d+,?\d+,?\d+)')
#extract the numbers only in the 'Area' column

In [29]:
final_df['Birth rate'] = final_df['Birth rate'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Birth rate' column

In [30]:
final_df['Coastline'] = final_df['Coastline'].str.extract('(\d+.?\d+)')
#extract the numbers only in the 'Coastline' column

In [31]:
final_df['Death rate'] = final_df['Death rate'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Death rate' column

In [32]:
final_df[['Agriculture','Industry','Services','None']]= final_df['GDP_composition'].str.split('\n',expand=True)
#split the 'GDP_composition' column into columns that contain the data about the GDP in agriculture, industry, and service
final_df = final_df.drop(['GDP_composition', 'None'],axis=1)
#drop the unecessary columns
final_df['Agriculture'] = final_df['Agriculture'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Agriculture' column
final_df['Industry'] = final_df['Industry'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Industry' column
final_df['Services'] = final_df['Services'].str.split('%',expand=True)
final_df['Services'] = final_df['Services'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Services' column

In [33]:
final_df[['GDP','a','b','c','d']] = final_df['GDP_PPP'].str.split('\n',expand=True)
#split the 'GDP_PPP' column so that I get the column that contains the data of the GDP recorded the most recently
final_df = final_df.drop(['GDP_PPP','a','b','c','d'],axis=1)
#drop the unnecessary data
final_df['GDP'] = final_df['GDP'].str.split('(',expand=True)
#keep only the numbers in the 'GDP' column

In [34]:
final_df[['health expenditures','a']] = final_df['Health expenditures'].str.split('%',expand=True)
#split the 'Health expenditures' column so that I get the column that contains the numbers before the % sign 
final_df = final_df.drop(['Health expenditures','a'],axis=1)
#drop the unnecessary columns

In [35]:
final_df['Infant mortality rate'] = final_df['Infant mortality rate'] .str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Infant mortality rate' column

In [36]:
final_df['Inflation rate'] = final_df['Inflation rate'].str.extract('(-?\d+.?\d?)')
#extract the numbers only in the 'Inflation rate' column

In [37]:
final_df[['num of Internet Users','Internet users %','a']] = final_df['Internet users'].str.split('\n',expand=True)
#split the 'Internet users' column so that I get the column that contain the data about the percentage of the population that are internet users
final_df = final_df.drop(['Internet users','num of Internet Users','a'],axis=1)
#drop the unnecessary columns
final_df['Internet users %'] = final_df['Internet users %'].str.extract('(\d+.?\d?)')
#extract the numbers only in the 'Internet users % of population' column

In [38]:
final_df[['def','literacy_total','literacy_male','literacy_female','d']]=final_df['Literacy'].str.split('\n',expand=True)
#split the 'Literacy' column into columns that contain data about the total literacy rate, the male literacy rate, and the female literacy rate
final_df['literacy_total'] = final_df['literacy_total'].str.extract('(\d+.?\d?)')
#extract only the numbers in the 'literacy_total' column
final_df['literacy_male'] = final_df['literacy_male'].str.extract('(\d+.?\d?)')
#extract only the numbers in the 'literacy_male' column
final_df['literacy_female'] = final_df['literacy_female'].str.extract('(\d+.?\d?)')
#extract only the numbers in the 'literacy_female' column
final_df = final_df.drop(['Literacy','def','d'],axis=1)
#drop the unnecessary columns

In [39]:
final_df['Net migration'] = final_df['Net migration'].str.extract('(-?\d+.?\d?)')
#extract only the numbers in the 'Net migration' column

In [40]:
final_df['Population'] = final_df['Population'].str.extract('(\d+,?\d+,?\d+)')
#extract only the numbers in the 'Population' column

In [41]:
final_df = final_df.fillna(0)
#replace the NaN values with the 0 value

In [42]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,"652,230",38.3,arid to semiarid; cold winters and hot summers,0,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123,0,"temperate; Mediterranean with hot, dry summers...",56.3,0,0,0,0,0,...,"15,700",0,0,0,0,0,0,0,0,0
2,Albania,"28,748",13.1,"mild temperate; cool, cloudy, wet winters; hot...",362,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,"2,381,741",23,"arid to semiarid; mild, wet winters with hot, ...",998,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199,22.9,"tropical marine, moderated by southeast trade ...",116,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0,31.3,0,0,0


In [43]:
final_df['Area'] = final_df['Area'].str.replace(',','').astype(float)
#turn the data in the 'Area' column into floats

In [44]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,0,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0,"temperate; Mediterranean with hot, dry summers...",56.3,0,0,0,0,0,...,"15,700",0,0,0,0,0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23,"arid to semiarid; mild, wet winters with hot, ...",998,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0,31.3,0,0,0


In [45]:
final_df['Birth rate'] = final_df['Birth rate'].astype('float')
#turn the data in the 'Birth rate' column into floats

In [46]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,0,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0,0,0,0,0,...,"15,700",0,0,0,0,0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0,31.3,0,0,0


In [47]:
final_df['Coastline'] = final_df['Coastline'].str.replace(',','').astype(float)
#turn the data in the 'Coastline' column into floats

In [48]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0,0,0,0,0,...,"15,700",0,0,0,0,0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0,31.3,0,0,0


In [49]:
final_df['Death rate'] = final_df['Death rate'].astype(float)
#turn the data in the 'Death rate' column into floats

In [50]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0,0,0,...,"15,700",0,0,0,0,0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0,31.3,0,0,0


In [51]:
final_df['health expenditures'] = final_df['health expenditures'].astype(float)
#turn the data in the 'health expenditures' column into floats

In [52]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0,0,0,...,"15,700",0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [53]:
final_df['Inflation rate'] = final_df['Inflation rate'].str.replace('%','').astype(float)
#turn the data in the 'Inflation rate' column into floats

In [54]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0,0.0,0,...,"15,700",0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [55]:
final_df['Infant mortality rate'] = final_df['Infant mortality rate'].astype(float)
#turn the data in the 'Infant mortality rate' column into floats

In [56]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0.0,0.0,0,...,"15,700",0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [57]:
final_df['Net migration'] = final_df['Net migration'].astype(float)
#turn the data in the 'Net migration' column into floats

In [58]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,"33,332,025",24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0.0,0.0,0,...,"15,700",0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,"3,038,594",21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,"40,263,711",13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,"54,194",27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [59]:
final_df['Population'] = final_df['Population'].str.replace(',','').astype(float)
#turn the data in the 'Population' column into floats

In [60]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,33332025.0,24%,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0.0,0.0,0,...,15700.0,0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,3038594.0,21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,40263711.0,13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,54194.0,27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [61]:
final_df['Agriculture'] = final_df['Agriculture'].str.replace('%','').astype(float)
#turn the data in the 'Agriculture' column into floats

In [62]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,NaN,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,33332025.0,24.0,21%,55,$62.32 billion,8.2,8.3,38.2,52%,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0.0,0.0,0,...,15700.0,0.0,0,0,0,0.0,0,0,0,0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,3038594.0,21.9,14.9,63.3,$32.65 billion,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,40263711.0,13.1,39%,47.9,$578.7 billion,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,54194.0,27.4,12.4,60.2,$711 million,0.0,31.3,0,0,0


In [63]:
final_df['Industry'] = final_df['Industry'].str.replace('%','').astype(float)
#turn the data in the 'Industry' column into floats

In [64]:
final_df['Services'] = final_df['Services'].str.replace('%','').astype(float)
#turn the data in the 'Services' column into floats

In [65]:
final_df['literacy_total'] = final_df['literacy_total'].str.replace('%','').astype(float)
#turn the data in the 'literacy_total' column into floats

In [66]:
final_df['literacy_male'] = final_df['literacy_male'].str.replace('%','').astype(float)
#turn the data in the 'literacy_male' column into floats

In [67]:
final_df['literacy_female'] = final_df['literacy_female'].str.replace('%','').astype(float)
#turn the data in the 'literacy_female' column into floats

In [68]:
final_df['Internet users %'] = final_df['Internet users %'].str.replace('%','').astype(float)
#turn the data in the 'Internet users % of population' column into floats

In [69]:
final_df['GDP'] = final_df['GDP'].str.replace('$','')
#discard the $ sign at the end of each string in the 'GDP' column
final_df['GDP'] = final_df['GDP'].str.replace(' ','')
#discard the space between the words in each string in the 'GDP' column
final_df['GDP'] = final_df['GDP'].replace('seeentryfortheWestBank',final_df.iloc[261,15])
#replace the value 'seeentryfortheWestBank' with the value of the cell at row 261 and column 15
final_df['GDP'] = final_df['GDP'].replace('NA',0)
#replace 'NA' values with the 0 value
final_df['GDP'] = final_df['GDP'].fillna(0)
#fill the NaN cells with the 0 value

In [70]:
final_df['GDP'] = final_df['GDP'].replace(r'[millionbilliontrillion]+$', '', regex=True).astype(float) * final_df['GDP'].str.extract(r'[\d\.]+([millionbilliontrillion]+)',expand=False).fillna(0).replace(['million','billion','trillion'], [10**6, 10**9, 10**12])
#in the 'GDP' column, the string is either of the form 'number + million' or 'number + billion' or 'number + trillion'.
#I would like to multiply the number with 10^6, 10^9, or 10^12 for million, billion, and trillion values. 
#Finally, I would turn every string into numbers

In [74]:
final_df = final_df.fillna(0)

In [75]:
final_df.head()

,Country,Area,Birth rate,Climate,Coastline,Death rate,Government type,Infant mortality rate,Inflation rate,Land use,...,Population,Agriculture,Industry,Services,GDP,health expenditures,Internet users %,literacy_total,literacy_male,literacy_female
0,Afghanistan,652230.0,38.3,arid to semiarid; cold winters and hot summers,0.0,13.7,presidential Islamic republic,112.8,-1.5,agricultural land: 58.1%\narable land 11.9%; p...,...,33332025.0,24.0,21.0,55.0,6.232000e+10,8.2,8.3,38.2,52.0,24.2
1,Akrotiri,123.0,0.0,"temperate; Mediterranean with hot, dry summers...",56.3,0.0,0,0.0,0.0,0,...,15700.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0
2,Albania,28748.0,13.1,"mild temperate; cool, cloudy, wet winters; hot...",362.0,6.7,parliamentary republic,12.3,1.9,agricultural land: 43.8%\narable land 22.7%; p...,...,3038594.0,21.9,14.9,63.3,3.265000e+10,5.9,63.2,97.6,98.4,96.9
3,Algeria,2381741.0,23.0,"arid to semiarid; mild, wet winters with hot, ...",998.0,4.3,presidential republic,20.3,4.8,agricultural land: 17.3%\narable land 3.1%; pe...,...,40263711.0,13.1,39.0,47.9,5.787000e+11,7.2,38.2,80.2,87.2,73.1
4,American Samoa,199.0,22.9,"tropical marine, moderated by southeast trade ...",116.0,4.8,presidential democracy; a self-governing terri...,8.5,2.1,agricultural land: 21.7%\narable land 13.3%; p...,...,54194.0,27.4,12.4,60.2,7.110000e+08,0.0,31.3,0.0,0.0,0.0


In [76]:
final_df.to_csv('2016 world_countries.csv', index=False)